<a href="https://colab.research.google.com/github/syslogg/classifier-algorithms/blob/master/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação Redes Neurais Multicamada (MLP)
- Hendy Rodrigues F. Silva (1510081)

In [159]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import warnings
import random as rnd
from random import randint

warnings.filterwarnings('ignore')

ds_derm = pd.read_csv('derm.csv')
ds_derm.drop(['age'], axis=1,inplace=True) # Descarta a IDADE
ds_derm.head()

,erythema,scaling,definite_borders,itching,koebner_phenomenon,polygonal_papules,follicular_papules,oral_mucosal_involvement,knee_and_elbow_involvement,scalp_involvement,...,focal_hypergranulosis,disappearance_of_the_granular_layer,vacuolisation_and_damage_of_basal_layer,spongiosis,saw-tooth_appearance_of_retes,follicular_horn_plug,perifollicular_parakeratosis,inflammatory_monoluclear_inflitrate,band-like_infiltrate,class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,0,3,0,0,0,1,0,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,1
2,2,1,2,3,1,3,0,3,0,0,...,2,0,2,3,2,0,0,2,3,3
3,2,2,2,0,0,0,0,0,3,2,...,0,3,0,0,0,0,0,3,0,1
4,2,3,2,2,2,2,0,2,0,0,...,2,2,3,2,3,0,0,2,3,3


In [0]:
last_col = ds_derm.columns[len(ds_derm.columns)-1]
classes = list(ds_derm[last_col].unique())
len_cols = len(ds_derm.columns) - 1

In [0]:
# One Hot Codification
# Codificacao usando arrays de zeros e um para cada classe
def one_hot_encoding(classes):
    cl_onehot = np.zeros((len(classes),len(classes)),dtype=int)
    np.fill_diagonal(cl_onehot,1)
    r = [(classes[i], cl) for i, cl in enumerate(cl_onehot)]
    return r

# Codifica as classes esperadas
def encode_expected(expected, encoded_class):
    return np.array([ list(filter(lambda e: e[0] == x, encoded_class))[0][1] for x in expected ])

# Codifica todas as classes do dataset
def encode_class(ds):
    return one_hot_encoding(pd.unique(ds.iloc[:,-1:].values.flatten()))

# Decodifica o resultado
def decode_result(encoded_class, value):
    if sum(value) != 1:
        value = list(encoded_class[randint(0,len(encoded_class)-1)][1])
    return list(filter(lambda x: list(x[1]) == value,encoded_class))[0][0]

In [0]:
# Classe para representar a camada oculta
class NeuronLayer():
    def __init__(self, number_of_neurons, number_of_inputs_per_neuron):
        self.synaptic_weights = 2 * np.random.random((number_of_inputs_per_neuron, number_of_neurons)) - 1


In [0]:
# Classe para representar a rede neural (por padrao foi feita com 2 camadas)

class NeuralNetwork():
    
    # Metodo constructor
    def __init__(self, layer1, layer2):
        self.layer1 = layer1
        self.layer2 = layer2
        
    # Função Sigmoide
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # Derivada da Sigmoide
    def __sigmoid_derivative(self, x):
        return x * (1 - x)
    
    # Treinar Rede - Utilização: Regra Delta
    def train(self, inputs_training, outputs_training, num_interation):
        for interate in range(0,num_interation):
            
            # Calcula o resultado do neuronio
            output_layer1, output_layer2 = self.__think(inputs_training)
            
            # Calcula o erro da layer 2
            layer2_error = outputs_training - output_layer2
            layer2_delta = layer2_error * self.__sigmoid_derivative(output_layer2)
            
            # Calcula o erro da layer 1
            layer1_error = layer2_delta.dot(self.layer2.synaptic_weights.T)
            layer1_delta = layer1_error * self.__sigmoid_derivative(output_layer1)
            
            # Quanto de ajuste vai ter no W das camadas 
            layer1_adjustment = inputs_training.T.dot(layer1_delta)
            layer2_adjustment = output_layer1.T.dot(layer2_delta)
            
            #Ajusta os pesos dos neuronios
            self.layer1.synaptic_weights += layer1_adjustment
            self.layer2.synaptic_weights += layer2_adjustment
            
            
            
    def __think(self, input_training):
        output_layer1 = self.__sigmoid(np.dot(input_training, self.layer1.synaptic_weights))
        output_layer2 = self.__sigmoid(np.dot(output_layer1, self.layer2.synaptic_weights))
        return output_layer1, output_layer2
    
    
    def predict(self, input_):
        h, out = self.__think(input_)
        result = [1 if o >= 0.5 else 0 for o in out]
        return result
        
    
    def print_weights(self):
        
        print('Camada 1:')
        print(self.layer1.synaptic_weights)
        print('Camada 2:')
        print(self.layer2.synaptic_weights)
    
    
            

In [0]:
def train(dataset_train, dataset_test):
    count_correct = 0
    count_incorrect = 0
    
    count_by_classes_correct = [0 for i in range(0,len(classes))]
    count_by_classes_incorrect = [0 for i in range(0,len(classes))]
    
    # Codifica as classes do dataset
    encoded_class = encode_class(dataset_train)
    
    # Rede neural com 2 Camadas, Uma com 16 Neuronios e Outra com 6 Neuronios.
    
    # Camada 1: Cria 16 Neuronios com 33 entradas (quantidade de inputs do dataset dermatology)
    l1 = NeuronLayer(16,len_cols)
    # Camada 2: Cria 12 Neuronios com 6 entradas vinda do outro neuronio (Camada de output)
    l2 = NeuronLayer(len(classes), 16)
    
    neural_network = NeuralNetwork(l1, l2)
    
    inputs = dataset_train.iloc[:,:-1].values
    outputs = dataset_train.iloc[:,-1:].values
    
    outputs_encoded = encode_expected(outputs,encoded_class)
    
    neural_network.train(inputs, outputs_encoded, 10000)
    
    for index, row in dataset_test.iterrows():
        
        tuple_t = list(row)
        tuple_t.pop()
        
        r = neural_network.predict(tuple_t) # Efetua o resultado pelo valor da rede
        
        result = decode_result(encoded_class, r)
        
        #Result
        if result == row[last_col]:
            count_correct += 1
            count_by_classes_correct[classes.index(result)] += 1
        else:
            count_incorrect += 1
            count_by_classes_incorrect[classes.index(result)] += 1
        
    return count_correct, count_incorrect, count_by_classes_correct, count_by_classes_incorrect

In [0]:
def seperate_ds_by_class(dataset, percentage):
    rds_train = pd.DataFrame()
    rds_test = pd.DataFrame()
    
    for c in classes:
        nds = dataset[dataset[last_col]==c]
        
        # Essa função pega o dataset e separa uma fração dele, e reordena
        ds_train = nds.sample(frac=percentage, random_state=randint(0,15100))
        # Pega o que sobrou do dataset de treino
        ds_test = nds.drop(ds_train.index) 
        
        rds_train = rds_train.append(ds_train)
        rds_test = rds_test.append(ds_test)
        
    rds_train = rds_train.reset_index() # Reiniciar indice
    rds_test = rds_test.reset_index() # Reiniciar indice

    rds_train.drop('index',1,inplace=True) # Retirar coluna index
    rds_test.drop('index',1,inplace=True) # Retirar coluna index
    
    return (rds_train, rds_test)

In [0]:
def run_nth(ds,percentage, number):
    percentages_correct = list()
    prob_correct_by_class = []
    
    for i in range(0,number):
        ds_train, ds_test = seperate_ds_by_class(ds,percentage)
        correct, incorrect, count_by_classes_correct, count_by_classes_incorrect = train(ds_train, ds_test)

        by_class = []
        for count_correct, count_incorrect in zip(count_by_classes_correct, count_by_classes_incorrect):
            if count_correct+count_incorrect != 0:
                by_class.append(count_correct/(count_correct+count_incorrect))
            else:
                by_class.append(0)
                
        prob_correct_by_class.append(by_class)
        percentages_correct.append(correct/(correct+incorrect))
        
    return (percentages_correct, prob_correct_by_class)

In [0]:
percents, prob_by_class = run_nth(ds_derm,0.8,30)

In [168]:
taxa_acerto_min=np.min(percents)
taxa_acerto_max=np.max(percents)
taxa_acerto_med=np.mean(percents)

print('Taxa de Acerto')
print('--------------')
print('Minimo: ' + str(taxa_acerto_min))
print('Máxima: ' + str(taxa_acerto_max))
print('Média: '+str(taxa_acerto_med))

Taxa de Acerto
--------------
Minimo: 0.05555555555555555
Máxima: 0.3055555555555556
Média: 0.19583333333333333


In [169]:
print('Taxa de Acerto Médio por classe')
print('-------------------------------')

ar_value = [ np.mean(m) for m in np.array(prob_by_class).transpose() ]

for i, _class in enumerate(ar_value):
    print('Classe \'' +  str(classes[i]) +'\' : ' + str(_class))

Taxa de Acerto Médio por classe
-------------------------------
Classe '2' : 0.12947723572723574
Classe '1' : 0.2973884938590821
Classe '3' : 0.16986279372502286
Classe '5' : 0.09794370590423221
Classe '4' : 0.12676483647071882
Classe '6' : 0.033972832722832726
